- leave running for 10+ hours to get ~50,000 tweets
- use keywords for scraping filter 
- try to figure something out relevant to content
- given sentiment analysis looks innaccurate - does it still correlate with favorable/unfavorable ranking (different from voting - how do you like them) Real Clear Politics does an average 
- how many pos/neg tweets per candidate --- compare against number of tweets, so could say ok trump has most tweets, but not all positive - compare 'who's winning on twitter'with poll data 
- can conclude candidate more popular on twitter than in polls, alt. is that twitter is good predictor for poll data 
- what are common topics ... topic clustering - does it vary by candidate? 

# First pass at exploring Twitter scraping
- used tweepy to scrape the data & output json file 

In [66]:
from pymongo import MongoClient
import json
import tweepy
from tweepy import Stream
from tweepy.streaming import StreamListener
import time
import tweetExplore_fns as fns
import importlib
importlib.reload(fns);



## organizing the data

In [67]:
#  import data 
# with open('raw_tweets10min.json') as data_file:
with open('../data/raw_tweetsfilter10min_poli.json') as data_file:
# with open('raw_tweets10min_loc.json') as data_file:
# with open('raw_tweets40min_bosSC.json') as data_file:
    rawdata = json.load(data_file)

# selects only tweets that have text 
tweet_list = fns.extract_tweets(rawdata)
        
#  extract indices for tweets that also have a fields of interest
coord_index_list = fns.extract_indices(tweet_list,"coordinates")

# extract tweets that also have a gps coordinate 
coord_tweet_list = fns.extract_tweets_fieldOfInterest(tweet_list,"coordinates")

print('N coordinates:',len(coord_index_list), 'N Tweets', len(tweet_list))

N coordinates: 10 N Tweets 10664


In [10]:
tweet_list[1]["text"]

"RT @AnnCoulter: Isn't Pope's attack on Trump, a presidential candidate, a violation of tax-free status? Tax the Catholic Church!"

## Trying out the textblob sentiment analysis

In [62]:
from textblob import TextBlob
candidates = ['trump','bush','clinton','sanders','cruz','kasich'] #track list
# for each candidate
for person,val in enumerate(candidates):
    person_sentiment_polarity = []
    person_tweetInd = []
    person_sentiment_subjectivity = []
    cur_person = candidates[person]
    # for each tweet
    for tweet,val in enumerate(tweet_list[:1]):
        curText = tweet_list[tweet]['text']
        cur_blob = TextBlob(curText)
        # check that the tweet contains mention of candidate
        if cur_blob.words.count(cur_person):
            cur_sentiment = cur_blob.sentiment
            # save sentiment polarity and subjectivity
            person_sentiment_polarity.append(cur_sentiment[0])
            person_sentiment_subjectivity.append(cur_sentiment[1])
            # save tweet index
            person_tweetInd.append(tweet)
        else:
            x = 1
    print(candidates[person], '  # tweets:',len(person_tweetInd), 'avg polarity:', )

trump   # tweets: 1 avg polarity:
bush   # tweets: 0 avg polarity:
clinton   # tweets: 0 avg polarity:
sanders   # tweets: 0 avg polarity:
cruz   # tweets: 0 avg polarity:
kasich   # tweets: 0 avg polarity:


In [60]:
print(cur_blob)
cur_sentiment = cur_blob.sentiment
polarity = cur_sentiment[0]
print(polarity)
type(polarity)
temp_list = []
temp_list.append(polarity)
print(temp_list)
# print(testl)
# print(person_sentiment_polarity.append(testl))
# type(cur_sentiment[0])
# print(cur_sentiment[0])

Go get this win Mr Trump  https://t.co/rYUxE2VmVg
0.8
[0.8]


In [ ]:
tweetBlob = TextBlob(joinedTexts)
candidates = ['Trump','Clinton','Kasich','Sanders','Bush','Cruz','Rubio']
storecount = [];
for i in candidates:
#     print(i)
    storecount.append(test.words.count(i))
fig = plt.gcf()
matplotlib.rc('xtick',labelsize = 25)
matplotlib.rc('ytick',labelsize = 20)
# matplotlib.rc('ylabel',labelsize = 20)
plt.ylabel('Number of Tweets')
ind = np.arange(len(candidates))
rect1 = plt.bar(ind, storecount, .3, color = 'blue')
plt.xticks(ind+.15, candidates);

## Plot geographical coordinates of tweets

In [ ]:
# This just lets the output of the following code samples
#  display inline on this page, at an appropriate size.
from pylab import rcParams
%matplotlib inline
rcParams['figure.figsize'] = (10,20)

In [ ]:
import matplotlib
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
# setup Lambert Conformal basemap.
m = Basemap(projection='lcc',width=12000000,height=9000000,
            resolution='l',lat_1=45.,lat_2=55,lat_0=50,lon_0=-107.)
#draw boundary, fill ocean color
m.drawmapboundary(fill_color='#CCFFFF')
# fill continents, set lake color
m.fillcontinents(color='#66CC99',lake_color='#CCFFFF') # '#6699CC'
# draw parallels and meridians. label only on edges of map
parallels = np.arange(0.,81,10.)
m.drawparallels(parallels,labels=[False,True,True,False])
meridians = np.arange(10.,351.,20.)
m.drawmeridians(meridians,labels=[True,False,False,True])
# drawstates, if no linewidth will be empty
m.drawstates(linewidth = .5)
m.drawcoastlines()
m.drawcountries()
# m.drawcounties(linewidth = 0.05, linestyle = 'solid', color = 'k', antialiased = 1)

i = 0
while i < len(coords):
    # get current coordinates 
    lon, lat = coords[i]
    # convert to map projection coords.
    # Note that lon,lat can be scalars, lists or numpy arrays.
    xpt,ypt = m(lon,lat)
    # convert back to lat/lon
    lonpt, latpt = m(xpt,ypt,inverse=True)
    m.plot(xpt,ypt,'ro')  # plot a blue dot there
    label = 'Hello'
    plt.Text(xpt+1000, ypt+5000, label)
    # put some text next to the dot, offset a little bit
    # (the offset is in map projection coordinates)
#     plt.text(xpt+100000,ypt+100000,'Boulder (%5.1fW,%3.1fN)' % (lonpt,latpt))
#     plt.text(xpt+100000,ypt+100000,'(%5.1fW,%3.1fN)' % (i,lonpt,latpt))
    i = i + 1
plt.show()

In [ ]:
rcParams['figure.figsize'] = (20,20)

 
# make sure the value of resolution is a lowercase L,
#  for 'low', not a numeral 1
# my_map = Basemap(projection='ortho', lat_0=50, lon_0=-100,
#               resolution='l', area_thresh=1000.0)
# my_map = Basemap(projection='robin', lat_0=50, lon_0=-100,
#               resolution='l', area_thresh=1000.0)
my_map = Basemap(projection='merc', lat_0=42.36, lon_0=-71.06,
              resolution='h', area_thresh=.01, 
              llcrnrlon=-71.3, llcrnrlat=42.1,
              urcrnrlon=-70.9, urcrnrlat=42.6) 
#               llcrnrlon=-71.098709, llcrnrlat=42.328093,
#               urcrnrlon=-71.012192, urcrnrlat=42.389994) 
my_map.drawcoastlines()
my_map.drawcountries()
my_map.fillcontinents(color='#66CC99')
my_map.drawstates(linewidth = .5)
my_map.drawmapboundary()
my_map.drawmeridians(np.arange(0, 360, 30))
my_map.drawparallels(np.arange(-90, 90, 30))
# my_map.drawcounties(linewidth = 0.5)

# lon = -135.3318
# lat = 57.0799
# x,y = my_map(lon, lat)
# my_map.plot(x, y, 'bo', markersize=12)
i = 0
while i < len(coords):
    # get current coordinates 
    lon, lat = coords[i]
    # convert to map projection coords.
    # Note that lon,lat can be scalars, lists or numpy arrays.
    xpt,ypt = my_map(lon,lat)
    # convert back to lat/lon
    lonpt, latpt = my_map(xpt,ypt,inverse=True)
    my_map.plot(xpt,ypt,'ro')  # plot a blue dot there
    label = 'Hello'
    plt.Text(xpt+1000, ypt+5000, label)
    # put some text next to the dot, offset a little bit
    # (the offset is in map projection coordinates)
#     plt.text(xpt+100000,ypt+100000,'Boulder (%5.1fW,%3.1fN)' % (lonpt,latpt))
#     plt.text(xpt+100000,ypt+100000,'(%5.1fW,%3.1fN)' % (i,lonpt,latpt))
    i = i + 1
plt.show()

## Display wordcloud for tweet text

In [ ]:

# no_urls_no_tags = " ".join([word for word in words.split()
#                             if 'http' not in word
#                                 and not word.startswith('@')
#                                 and word != 'RT'
#                             ])

# join tweets into a single string
joinedTexts = " ".join(str(x) for x in texts)

In [ ]:
from wordcloud import WordCloud, STOPWORDS
newSTOP = STOPWORDS
# add stopwords to exclude from wordcloud - 
newSTOP.update(('co','https','t','RT','R','T','-RT','RT@'))

#generate wordcloud
wordcloud = WordCloud(font_path='/anaconda/envs/eq_env/lib/python3.5/site-packages/matplotlib/mpl-data/fonts/ttf/cmmi10.ttf',
                          stopwords=newSTOP,
                          background_color='white',
                          width=1700,
                          height=1400
                         ).generate(joinedTexts)
# display wordcloud
plt.imshow(wordcloud)
plt.axis('off')
plt.show();

## Trying out the document clustering.... http://brandonrose.org/clustering

In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
# import mpld3

In [ ]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [ ]:

# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [ ]:
#not super pythonic, no, not at all.
#use extend so it's a big flat list of vocab
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in texts:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [ ]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print( 'there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(texts) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

In [ ]:
terms = tfidf_vectorizer.get_feature_names()


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)
print(dist)

In [ ]:
from sklearn.cluster import KMeans

num_clusters = 5

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

In [ ]:
from sklearn.externals import joblib

#uncomment the below to save your model 
#since I've already run my model I am loading from the pickle

joblib.dump(km,  'doc_cluster.pkl')

km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [ ]:
clusters